In [1]:
%cd ..

/home/ubuntu/SPVD_Lightning


In [2]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40
path = "./data/ModelNet40/pointclouds"

tr, te = get_dataloaders(path)
te = ModelNet40(path)

Loading data: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  5.45category/s]


In [9]:
from utils.visualization import visualize_notebook
import torch

t = torch.stack([te[i]['pc'] for i in range(32)])
visualize_notebook(t, x_offset=2.5, y_offset=2.5, point_size=0.025)

Output()

In [11]:
from models import *

m = SPVD_S()

lr = 1e-4
model = DiffusionBase(m, lr=lr)

model_state_dict = torch.load("checkpoints/ModelNet40/SPVD.ckpt", weights_only=True)['state_dict']
model.load_state_dict(model_state_dict)
model = model.cuda().eval()

from utils.schedulers import DDPMSparseSchedulerGPU
ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

In [ ]:
preds = ddpm_sched.sample(model.cuda(), 16, 2048)
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)